In [3]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np

# Initialize camera and FaceMesh
cam = cv2.VideoCapture(0)

if not cam.isOpened():
    print("Error: Could not open camera.")
    exit()

face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
screen_w, screen_h = pyautogui.size()

# Cooldown for left click
blink_cooldown = 1  # 1 second cooldown
last_left_click_time = 0

# Variables for head movement tracking
prev_nose_x, prev_nose_y = None, None
sensitivity = 6  # Increased sensitivity for better accuracy

# Eye blink detection variables
eye_closure_threshold = 0.015  # Tune this based on testing
blink_consecutive_frames = 3  # Number of consecutive frames needed for a valid blink
blink_counter = 0  # To count frames when eyes are closed

while True:
    ret, frame = cam.read()
    if not ret:
        print("Error: Failed to capture image")
        break  # Exit loop if frame is empty

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    if landmark_points:
        landmarks = landmark_points[0].landmark

        # Cursor Movement using Nose Landmark (Head Movement Control)
        nose = landmarks[1]  # Nose tip landmark
        nose_x, nose_y = int(nose.x * frame_w), int(nose.y * frame_h)

        if prev_nose_x is not None and prev_nose_y is not None:
            dx = (nose_x - prev_nose_x) * sensitivity
            dy = (nose_y - prev_nose_y) * sensitivity
            pyautogui.moveRel(dx, dy)

        prev_nose_x, prev_nose_y = nose_x, nose_y

        # Eye Blink Detection (Both Eyes for Left Click)
        left_eye = [landmarks[145], landmarks[159]]
        right_eye = [landmarks[374], landmarks[386]]

        left_eye_ratio = left_eye[0].y - left_eye[1].y
        right_eye_ratio = right_eye[0].y - right_eye[1].y

        if left_eye_ratio < eye_closure_threshold and right_eye_ratio < eye_closure_threshold:
            blink_counter += 1
        else:
            blink_counter = 0  # Reset counter if eyes are open

        # Check if blink is sustained for required frames
        if blink_counter >= blink_consecutive_frames:
            if time.time() - last_left_click_time > blink_cooldown:
                pyautogui.click()  # Left Click
                last_left_click_time = time.time()
                blink_counter = 0  # Reset counter after clicking

    cv2.imshow('Eye Controlled Mouse', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cam.release()
cv2.destroyAllWindows()


Error: Failed to capture image
